In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q keras

**IMPORTING THE LIBRARIES**

In [3]:
import keras


In [4]:
import tensorflow

In [5]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt



In [6]:
imageSize = [224,224]

In [7]:
train_path ='/content/drive/MyDrive/Data/Test/test-20221004T084138Z-001/test'
test_path ='/content/drive/MyDrive/Data/Train/train-20221004T084544Z-001/train'

**LOADING THE MODEL**

In [9]:
vgg = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

**ADDING FLATTEN LAYERS**

In [10]:
for layer in vgg.layers:
  layer.trainable = False

In [11]:
x = Flatten()(vgg.output)

**ADDING OUTPUT LAYER**

In [12]:
prediction = Dense(17, activation='softmax')(x)

**CREATING A MODEL OBJECT**

In [13]:
model = Model(inputs=vgg.input, outputs=prediction)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

**CONFIGURING THE LEARNING PROCESS**

In [16]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'], run_eagerly=True
)

**IMPORT IMAGEDATAGENERATOR**

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**CONFIGURE IMAGEDATAGENERATOR CLASS**

In [18]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

**APPLY IMAGEDATAGENERATOR**

In [19]:
training_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Data/Train/train-20221004T084544Z-001/train',
                                                target_size = (224, 224),
                                                batch_size = 32,
                                                class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Data/Test/test-20221004T084138Z-001/test',
                                             target_size = (224, 224),
                                            batch_size =32,
                                            class_mode = 'categorical')

Found 655 images belonging to 17 classes.
Found 183 images belonging to 17 classes.


**TRAIN THE MODEL**

In [20]:
import sys


In [21]:

r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=100,
    steps_per_epoch=len(training_set)//3,
    validation_steps=len(test_set)//3)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/100
7/7 [==============================] - 131s 17s/step - loss: 4.0963 - accuracy: 0.0966 - val_loss: 3.3283 - val_accuracy: 0.0781
Epoch 2/100
7/7 [==============================] - 65s 10s/step - loss: 3.2662 - accuracy: 0.1652 - val_loss: 2.8506 - val_accuracy: 0.2656
Epoch 3/100
7/7 [==============================] - 34s 5s/step - loss: 2.6141 - accuracy: 0.2589 - val_loss: 2.5207 - val_accuracy: 0.2812
Epoch 4/100
7/7 [==============================] - 24s 3s/step - loss: 2.3623 - accuracy: 0.2902 - val_loss: 2.0093 - val_accuracy: 0.4219
Epoch 5/100
7/7 [==============================] - 12s 2s/step - loss: 1.7984 - accuracy: 0.4866 - val_loss: 1.8098 - val_accuracy: 0.3906
Epoch 6/100
7/7 [==============================] - 10s 1s/step - loss: 1.6317 - accuracy: 0.5402 - val_loss: 1.3990 - val_accuracy: 0.6406
Epoch 7/100
7/7 [==============================] - 6s 899ms/step - loss: 1.5545 - accuracy: 0.5749 - val_loss: 1.6147 - val_accuracy: 0.5469
Epoch 8/100
7/7 [=====

**SAVE THE MODEL**

In [22]:
model.save('vgg-16-nail-disease.h5')

**TEST THE MODEL**

In [23]:
from tensorflow.keras.models import load_model


In [24]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

In [25]:
model=load_model('vgg-16-nail-disease.h5')

In [26]:
img=image.load_img(r'/content/drive/MyDrive/detection/Dataset/test/Darier_s disease/1.PNG',target_size=(224,224))

In [27]:
img = image.img_to_array(img)

In [28]:
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))

In [29]:
img = preprocess_input(img)

In [30]:
pred = model.predict(img)

1/1 [==============================] - 1s 847ms/step


In [31]:
output=np.argmax(model.predict(img), axis=1)

1/1 [==============================] - 0s 36ms/step


In [32]:
index=['Darier_s disease', 'Muehrck-e_s lines', 'aloperia areata', 'beau_s lines', 'bluish nail',
        'clubbing','eczema','half and half nailes(Lindsay_s nails)','koilonychia','leukonychia',
        'onycholycis','pale nail','red lunula','splinter hemmorrage','terry_s nail','white nail','yellow nails']
        
        

In [33]:
result = str(index[output[0]])

In [34]:
result

'Darier_s disease'